# 前処理

MeCabを用いた分かち書きやscreen_name,リンクの除去など

In [1]:
import re
import MeCab
import pickle
from tqdm import tqdm
from joblib import Parallel, delayed

In [2]:
def tokenizer_jp(sentence):
  return MeCab.Tagger("-Owakati -d /usr/lib/mecab/dic/mecab-ipadic-neologd").parse(sentence)

def creaning(sentence):
  sentence = re.sub(r"@([A-Za-z0-9_]+)", "", sentence)
  sentence = re.sub(r'https?:\/\/.*', "", sentence)
  return sentence

def preprocess(sentence):
    return tokenizer_jp(creaning(sentence))

In [3]:
corpus_path = "corpus/"
filename_raw = corpus_path + "total.txt"
filename_input = corpus_path + "input.pkl"
filename_output = corpus_path + "output.pkl"

In [4]:
%%time
with open(filename_raw,"r",encoding="utf-8")as f:
    print("Now Loading File...\n")
    data = f.readlines()
    
    print("Now Parsing...\n")
    data = Parallel(n_jobs=-1)([delayed(creaning)(_s) for _s in data])
    
    uttarances = data[0::2]
    responses = data[1::2]
    
    print("Finished.\n")

Now Loading File...

Now Parsing...

Finished.

CPU times: user 1min 37s, sys: 26.1 s, total: 2min 3s
Wall time: 2min 3s


In [5]:
p = re.compile(r"^\s*$")

_tmp = [[_u,_r] for i, (_u, _r) in enumerate(zip(data[0::2],data[1::2])) if not re.match(p, _u) and not re.match(p, _r)]
uttarances = [_t[0] for _t in _tmp]
responses = [_t[1] for _t in _tmp]

print("Saving")
with open(filename_input, "wb")as f:
    pickle.dump(uttarances, f)
    
with open(filename_output, "wb")as f:
    pickle.dump(responses, f)

Saving
